In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [6]:
def get_soup(url):
    # Make a GET request to the website
    response = requests.get(
        url,
        headers={'User-Agent': 'Mozilla/5.0'}
    )

    return BeautifulSoup(response.content, "lxml")

In [7]:
season_pages = {
    2020:'https://2kdb.net/collections/20',
    2021:'https://2kdb.net/collections/21',
    2022:'https://2kdb.net/collections/22',
    2023:'https://2kdb.net/collections/23',
}

In [ ]:
verbose = True

for season_year, url in season_pages.items():

    if verbose: print(season_year)

    rating_all_players = []

    soup = get_soup(url)

    team_links = soup.find_all('a', {"class": 'hover:bg-navbar rounded-xl p-4'})

    for link in team_links:
        url_team = 'https://2kdb.net'+link['href']
        team_name = link['title']
        
        if verbose: print(team_name)

        # Make a GET request to the website
        soup_team = get_soup(url_team)
        players_team = soup_team.find_all('a')
        for player in players_team:
            if (re.search('player',player['href']) is not None and player.text==''):
                time.sleep(1)
                url_player = 'https://2kdb.net'+player['href']

                soup_player = get_soup(url_player)
                # print(soup_player)
                name = soup_player.find('span',{'class':'flex items-center gap-2 font-bold text-3xl'}).text
                player_name = re.split('\d',name)[0]
                
                if verbose: print(player_name)

                stats = soup_player.find_all('div',{'class':'stats'})

                rating_dict = {}
                rating_dict['Season Year'] = season_year
                rating_dict['Team'] = team_name
                rating_dict['Player'] = player_name
                for rating_whole in stats:
                    rating_list = rating_whole.find_all('span')
                    # print(rating_list[0].text,rating_list[1].text)
                    rating_dict[rating_list[1].text] = rating_list[0].text
                rating_all_players.append(rating_dict)


In [ ]:
pd.DataFrame(rating_all_players).to_csv('ratings_NBA2k_2020_2023.csv')

pd.DataFrame(rating_all_players).head()